In [0]:
import requests
from bs4 import BeautifulSoup
import json 
import numpy as np

import sklearn as sk
import sklearn.neural_network
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#Jordi
class Task:
    id = ""
    completed = False
    category = 0
    reportId = ""
    duration = 0
    sla = 0

    def __init__(self,id, completed, category, reportId, duration,sla):
        self.id = id
        self.completed = completed
        self.category = category
        self.reportId = reportId
        self.duration = duration
        self.sla = sla

    def toString(self):
        return str(self.id) + " " + str(self.completed) + " " + self.category.toString() + " " + self.reportId + " " + str(self.duration) + " " + str(self.sla)

class Category:
    cat_type = 0
    description = ""
    stressLevel = 0

    def __init__(self, type, description, stressLevel):
        self.cat_type = type
        self.description = description
        self.stressLevel = stressLevel

    def toString(self):
        return str(self.cat_type) + " " + self.description + " " + str(self.stressLevel)

class Scenario:
    scenarioId = ""
    tasks = 0
    
    def __init__(self, scenarioId, tasks):
        self.scenarioId = scenarioId
        self.tasks = tasks
        
class Evaluations:
    evaluations = 0
    
    def __init__(self, evaluations):
        self.evaluations = evaluations
        
class ScenarioGuess:
    scenario = 0
    taskId = ""
    def __init__(self, scenario, taskId):
        self.scenario = scenario
        self.taskId = taskId
        
class Evaluation:
    scenarioId = ""
    taskId = ""
    
    def __init__(self, scenarioId, taskId):
        self.scenarioId = scenarioId
        self.taskId = taskId


In [0]:
sets = ""
n = 0
with open('/content/drive/My Drive/Udl/HackEps/Api/api-backup1.json', 'r') as json_file:
  for scenario in json_file:
    sets += scenario
   
json_ = json.loads(sets)

In [0]:
#Joel
sets = ""
n = 0
with open('/content/drive/My Drive/Api/api-backup1.json', 'r') as json_file:
  for scenario in json_file:
    sets += scenario
   
json_ = json.loads(sets)


In [0]:

id_to_encode = []

n = 0
for scenarion in json_['assignations']:
    tasksInScenario = scenarion['scenario']['tasks']
    for task in tasksInScenario:
      id_to_encode.append(task['id']) 
    if n == 20:#cambiar a 20
        break
    n+=1    
encoder2 = preprocessing.LabelEncoder()
encoder2.fit(id_to_encode)


In [0]:
n = 0
y_scenario = []
for scenarion in json_['assignations']:
  scenario = scenarion['scenario']
  s = requests.post('http://18.184.40.57:9010/scenario?teamKey=lmao', json={"scenario":scenario , "taskId":""}).json()
  y_scenario.append(s['value'])
  if n == 20:
    break
  n += 1

print(y_scenario)

In [0]:
#alejandro 
n = 0
x_train = []
y_train = []
i = 0
for scenarion in json_['assignations']:
    tasksInScenario = scenarion['scenario']['tasks']
    j = 0
    for task in tasksInScenario:
      task_id = task['id']
      if task_id == encoder2.inverse_transform([y_scenario[i]])[0]:
        print(task_id)
        print(encoder2.inverse_transform([y_scenario[i]])[0])
        y_train.append(j * 7)
        j +=1
    if n == 20:
        break
    i+=1        
    n+=1
print(y_train) 
#print(encoder2.inverse_transform([273]))


In [0]:
#Joel
n = 0
x_train = []
for scenarion in json_['assignations']:
  tasksInScenario = scenarion['scenario']['tasks']
  task_list = []
  for task in tasksInScenario:
    task_id = encoder2.transform([task['id']])[0]
    if task['completed']==True:
        task_completed = 1
    else:
      task_completed = 0
    category_type = float(task['category']['type'])
    category_stressLevel = float(task['category']['stressLevel'])
    task_reportId = float(task['reportId'])
    task_duration = float(task['duration'])
    task_sla = float(task['sla'])
    if len(task_list)==0:
      task_list = [task_id, task_completed, category_type, category_stressLevel, task_reportId, task_duration, task_sla]
    else:
      task_list = task_list + [task_id, task_completed, category_type, category_stressLevel, task_reportId, task_duration, task_sla]
  x_train.append(task_list)    
  if n==20:
    break
  else:
    n+=1


In [0]:

y_train = []
j = 0
i = 0

for p in x_train:
  while i < len(p) or i + 7 < len(p):
   
    if p[i] == encoder2.transform([y_scenario[j]]):
     y_train.append(i)
     break
    i+=7
  i = 0   
  j += 1   

In [0]:
#Implementacion red neuronal
lr = 0.01           # learning rate
nn = [2, 16, 8, 1]  # número de neuronas por capa.

# Creamos el objeto del modelo de red neuronal multicapa.
prediction_model = sk.neural_network.MLPRegressor(solver='sgd', 
                                     learning_rate_init=lr, 
                                     hidden_layer_sizes=tuple(nn[1:]),
                                     verbose=True,
                                     n_iter_no_change=1000,
                                     batch_size = 64)

prediction_model.fit(x_train,y_scenario)

In [0]:
model = RandomForestClassifier(n_estimators=5000, max_depth=1000)
model.fit(x_train, y_train)

In [0]:
r = requests.get('http://18.184.40.57:9010//evaluation?teamKey=lmao')
test = r.json()[0]

In [0]:
n = 0
x_set = []
for task in test['tasks']:
    task_id = 0
    if task['completed']==True:
        task_completed = 1
    else:
      task_completed = 0
    category_type = float(task['category']['type'])
    category_stressLevel = float(task['category']['stressLevel'])
    task_reportId = float(task['reportId'])
    task_duration = float(task['duration'])
    task_sla = float(task['sla'])
    if len(task_list)==0:
      task_list = [task_id, task_completed, category_type, category_stressLevel, task_reportId, task_duration, task_sla]
    else:
      task_list = task_list + [task_id, task_completed, category_type, category_stressLevel, task_reportId, task_duration, task_sla]
    x_set.append(task_list)    
    if n==20:
     break
    else:
     n+=1
print(len(x_set))

In [0]:
print(model.predict(x_set[0]))